In [1]:
%env PYTHONWARNINGS=ignore

import warnings
warnings.filterwarnings("ignore")

env: PYTHONWARNINGS=ignore


In [2]:
from dask_cuda import LocalCUDACluster
from dask.distributed import Client
from nemo_curator import DomainClassifier
from nemo_curator.datasets import DocumentDataset
#from nemo_curator.utils.distributed_utils import start_dask_gpu_local_cluster

In [3]:
cluster = LocalCUDACluster(rmm_async=True, rmm_pool_size="1GB")
client = Client(cluster)

In [4]:
labels = [
    "Adult",
    "Arts_and_Entertainment",
    "Autos_and_Vehicles",
    "Beauty_and_Fitness",
    "Books_and_Literature",
    "Business_and_Industrial",
    "Computers_and_Electronics",
    "Finance",
    "Food_and_Drink",
    "Games",
    "Health",
    "Hobbies_and_Leisure",
    "Home_and_Garden",
    "Internet_and_Telecom",
    "Jobs_and_Education",
    "Law_and_Government",
    "News",
    "Online_Communities",
    "People_and_Society",
    "Pets_and_Animals",
    "Real_Estate",
    "Science",
    "Sensitive_Subjects",
    "Shopping",
    "Sports",
    "Travel_and_Transportation",
]

model_file_name = "/home/nfs/syurick/LLM_domain_classifier_inference/GoogleDebertaAgree_v3b_bce_maxlen512_bs64_noRef_best.pth"
# Input can be a string or list
input_file_path = "/raid/vjawa/prospector-lm/subset_CC-MAIN-2023-14_english"
output_file_path = "/raid/vjawa/output_subset_CC-MAIN-2023-14_english"


In [5]:
%%time

input_dataset = DocumentDataset.read_json(
    input_file_path, backend="cudf", add_filename=True
)

domain_classifier = DomainClassifier(
    model_file_name=model_file_name,
    labels=labels,
    batch_size=1024,
)
result_dataset = domain_classifier(dataset=input_dataset)
result_dataset.to_json(output_file_dir=output_file_path, write_to_filename=True)

Reading 50 files


Starting domain classifier inference


GPU: 0, Part: 0: 100%|██████████| 9729/9729 [00:31<00:00, 309.69it/s]


2024-05-09 01:05:38,916 - tornado.application - ERROR - Uncaught exception GET /status/ws (10.2.226.46)
HTTPServerRequest(protocol='http', host='10.120.104.12:8787', method='GET', uri='/status/ws', version='HTTP/1.1', remote_ip='10.2.226.46')
Traceback (most recent call last):
  File "/datasets/vjawa/miniconda3/envs/NeMo-Curator-env-2/lib/python3.10/site-packages/tornado/web.py", line 1790, in _execute
    result = await result
  File "/datasets/vjawa/miniconda3/envs/NeMo-Curator-env-2/lib/python3.10/site-packages/tornado/websocket.py", line 273, in get
    await self.ws_connection.accept_connection(self)
  File "/datasets/vjawa/miniconda3/envs/NeMo-Curator-env-2/lib/python3.10/site-packages/tornado/websocket.py", line 863, in accept_connection
    await self._accept_connection(handler)
  File "/datasets/vjawa/miniconda3/envs/NeMo-Curator-env-2/lib/python3.10/site-packages/tornado/websocket.py", line 946, in _accept_connection
    await self._receive_frame_loop()
  File "/datasets/vjaw

Writing to disk complete for 50 partitions
CPU times: user 39.5 s, sys: 27.2 s, total: 1min 6s
Wall time: 3min 14s


In [8]:
output_dataset = DocumentDataset.read_json(output_file_path, backend="cudf", add_filename=True)

Reading 50 files


In [9]:
output_file_path

'/raid/vjawa/output_subset_CC-MAIN-2023-14_english'

In [10]:
output_dataset.df

,adlr_id,filename,labels,langid_score,language,source_id,text,url,warc_id
npartitions=50,,,,,,,,,
,object,object,object,float64,object,object,object,object,object
,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...


In [11]:
output_dataset.df.head(10)

,adlr_id,filename,labels,langid_score,language,source_id,text,url,warc_id
0,cc-2023-14-0001622299,crawl-data-CC-MAIN-2023-14-segments-1679296943...,Jobs_and_Education,0.946693,EN,crawl-data-CC-MAIN-2023-14-segments-1679296943...,Neighborhood Street Fund Application: plans fo...,http://12thaveseattle.com/blog/2013/02/11/12th...,6dd74af8-669e-4aaf-b5f8-e2a44f03574b
1,cc-2023-14-0001622300,crawl-data-CC-MAIN-2023-14-segments-1679296943...,Computers_and_Electronics,0.918942,EN,crawl-data-CC-MAIN-2023-14-segments-1679296943...,"Main navigation\n\nProject Assistance, Managem...",http://1kenthomas.com/slides/drupal-project-as...,eb065ae1-4737-4557-b040-96a1ecf67db4
2,cc-2023-14-0001622301,crawl-data-CC-MAIN-2023-14-segments-1679296943...,Autos_and_Vehicles,0.937426,EN,crawl-data-CC-MAIN-2023-14-segments-1679296943...,RENDI AUTO\n\nBMW X5\n\nPeriod\n\n-\n\nName\n\...,http://1rendiauto.ee/en/autod-2/?auto_id=13,9f69f52e-350d-4677-8609-d7f3c8d759c6
3,cc-2023-14-0001622302,crawl-data-CC-MAIN-2023-14-segments-1679296943...,Sensitive_Subjects,0.976790,EN,crawl-data-CC-MAIN-2023-14-segments-1679296943...,Now based on multiple underwater UFO encounter...,http://24newstodays.com/2022/12/23/weird-encou...,256174a9-506e-4b23-9580-de10e3ab4590
4,cc-2023-14-0001622303,crawl-data-CC-MAIN-2023-14-segments-1679296943...,Jobs_and_Education,0.906984,EN,crawl-data-CC-MAIN-2023-14-segments-1679296943...,We are commited to providing the highest level...,http://434caaeea2929142-u.edu-newsletters.com/...,f87fd3c6-1450-4260-93e5-bed50ea608e4
5,cc-2023-14-0001622304,crawl-data-CC-MAIN-2023-14-segments-1679296943...,Sports,0.977782,EN,crawl-data-CC-MAIN-2023-14-segments-1679296943...,football\n\nThe football season in Bulgaria ha...,http://4liberty.eu/tag/football/,933d5938-d5b7-4745-ba69-db153fb25173
6,cc-2023-14-0001622305,crawl-data-CC-MAIN-2023-14-segments-1679296943...,Pets_and_Animals,0.942719,EN,crawl-data-CC-MAIN-2023-14-segments-1679296943...,"As leaders of the No Kill Movement on Maui, we...",http://9thlifehawaii.org/site/Spay-amp-Neuter-...,c1a83c12-0fbb-4fc9-8c68-4a109d71048e
7,cc-2023-14-0001622306,crawl-data-CC-MAIN-2023-14-segments-1679296943...,Internet_and_Telecom,0.948170,EN,crawl-data-CC-MAIN-2023-14-segments-1679296943...,The IP address for this domain may have change...,http://a1levelingcleveland.com/cgi-sys/default...,48829d73-dc4b-4423-8dd5-db32ac6a4349
8,cc-2023-14-0001622307,crawl-data-CC-MAIN-2023-14-segments-1679296943...,Food_and_Drink,0.971278,EN,crawl-data-CC-MAIN-2023-14-segments-1679296943...,"A\nvegetarian since the age of 15, Abbey Levin...",http://abbeysvegetarianrecipes.com/abbey.html,223a469b-6001-4820-aa2d-7eadb6afa6c4
9,cc-2023-14-0001622308,crawl-data-CC-MAIN-2023-14-segments-1679296943...,Online_Communities,0.864449,EN,crawl-data-CC-MAIN-2023-14-segments-1679296943...,"Search This Blog\n\nDisclaimer\n\nThe owners, ...",http://abeckslife.blogspot.com/2011/12/woodwri...,9c57d2eb-dd01-4cc6-a473-7ae30ba26e3e
